<a href="https://colab.research.google.com/github/Sajupikachu/HTML/blob/main/Gen_ai_hands_on.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 # We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-04-25 06:50:40--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-04-25 06:50:40 (17.1 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [5]:
# Hey! I'm importing the PyTorch library, the core of this entire neural network setup. Without me, nothing's going to work!
import torch

# I'm importing the neural network module from PyTorch. Without this, you can't build layers or define the architecture of your model.
import torch.nn as nn

# I'm bringing in all the useful activation functions and loss functions. Forget me, and you'll be stuck manually defining basic operations like ReLU, softmax, and cross-entropy!
from torch.nn import functional as F

# Yo! I'm the batch size, handling 16 samples at a time, too high might blow up your memory, too low and your training could take forever.(HYPER_PARAMETRS)
batch_size = 16

# I'm block_size, controlling how much context (past tokens) I give to the model. Set me too small and your model will be short-sighted, too large and I'll be a memory hog.(HYPER_PARAMETRS)
block_size = 32

# I decide how long we're going to train. 5000 iterations! If you cut me off too early, the model won't learn enough. Too long, and you're wasting time.(HYPER_PARAMETRS)
max_iters = 5000

# Every 100 steps, I make the model pause and reflect (evaluate). If I'm too frequent, you'll slow things down. Too rare, and you might miss important signals.(HYPER_PARAMETRS)
eval_interval = 100

# I'm the learning rate! Too high and your model will be jittery, too low and it’ll learn at a snail's pace.(HYPER_PARAMETRS)
learning_rate = 1e-3

# I decide if we're using the power of a GPU (if available) or sticking with a CPU. If I mess this up, well... you’re gonna have a bad time!
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# I control how often we check validation loss during evaluation. More of me means better estimation, but also more time wasted.
eval_iters = 200

# I'm the size of the embedding. Bigger embeddings mean more expressive power, but go too big and you'll run out of memory.
n_embd = 64

# I'm the number of attention heads. More heads, more perspectives in attention, but too many and things will get chaotic (and slow).
n_head = 4

# I'm the number of layers in the Transformer. If I’m too shallow, the model won’t be deep enough to learn complex stuff. Too deep, and it’s gonna overfit.
n_layer = 4

# I drop out some neurons randomly during training to prevent overfitting. Set me too high and you’ll lose important info, too low and you'll overfit like crazy.
dropout = 0.0

torch.manual_seed(1337)  # I'm a seed to make things reproducible. Without me, good luck replicating your results!

# I’m here to read the Shakespeare dataset. If I fail, you'll have no data to train on and everything will break.
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# I’m the unique characters in the dataset, and if I don’t get sorted right, your entire encoding/decoding scheme will be messed up.
chars = sorted(list(set(text)))
vocab_size = len(chars)  # I’m the vocabulary size. If you miscalculate me, good luck generating meaningful text!

# These guys map characters to integers and back. Without me, the model can’t process text at all.
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

# I’m the encoder, translating characters into their integer equivalents. If I don’t work, nothing gets fed to the model!
encode = lambda s: [stoi[c] for c in s]

# I’m the decoder, turning numbers back into text. If I mess up, your output will be gibberish, not beautiful text.
decode = lambda l: ''.join([itos[i] for i in l])

# I'm turning the whole Shakespeare text into a big list of integers. Forget me and your model will be reading... nothing.
data = torch.tensor(encode(text), dtype=torch.long)

# I’m splitting the data into training (90%) and validation (10%). If I get this split wrong, your model could overfit or underfit.
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

# I'm responsible for creating mini-batches of data. No batching means you’re doing this slow and inefficiently.
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))  # Random start points for each sample in the batch.
    x = torch.stack([data[i:i+block_size] for i in ix])        # Input sequences for the model.
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])    # Target sequences (next token).
    x, y = x.to(device), y.to(device)  # Moving everything to GPU if available.
    return x, y

# I handle loss estimation without gradients. Without me, you'd be overworking the GPU and wasting memory.
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()  # I switch the model into evaluation mode (no dropout, etc.). Forget me, and your eval won’t be accurate.
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)  # A bunch of zeros to store losses for several iterations.
        for k in range(eval_iters):
            X, Y = get_batch(split)      # Get a batch.
            logits, loss = model(X, Y)   # Get the predictions and loss.
            losses[k] = loss.item()      # Store the loss.
        out[split] = losses.mean()       # Average the losses.
    model.train()  # Don’t forget to switch back to training mode. If you forget me, the model will stay stuck in eval mode!
    return out

# I'm a single head of attention. Forget me, and the model can't "focus" on different parts of the input sequence.
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)  # The "key" in attention. Without it, I can’t look up the relevant info.
        self.query = nn.Linear(n_embd, head_size, bias=False)  # The "query" in attention. If I mess up, I won’t know what to attend to.
        self.value = nn.Linear(n_embd, head_size, bias=False)  # The "value" being attended to. Lose me and the output's meaningless.

        # I’m a lower triangular matrix for causal attention (no peeking into the future!). Forget me, and the model will cheat!
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)  # Dropout layer. If you don’t use me right, the model might overfit.

    def forward(self, x):
        B, T, C = x.shape  # Batch size, time steps, and channels (embedding size).
        k = self.key(x)   # Compute the keys. Forget this and attention breaks.
        q = self.query(x) # Compute the queries. Without me, the model doesn’t know what to focus on.

        # Attention weights! No queries or keys = no attention. No scaling = unstable attention.
        wei = q @ k.transpose(-2, -1) * C**-0.5

        # I mask future positions, so the model only attends to past or present. Without me, it's cheating by looking ahead.
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))

        wei = F.softmax(wei, dim=-1)  # Normalize to get attention probabilities. Without this, your attention’s all over the place.
        wei = self.dropout(wei)  # Drop out some weights to prevent overfitting.

        # Compute the weighted sum of values. Without values, the model can't return any meaningful output.
        v = self.value(x)
        out = wei @ v        # Combine attention weights with the values.
        return out

# Multi-head attention: multiple heads, more perspectives! If you set me wrong, you'll lose parallelism or attention.
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        # I’m a bunch of heads that run attention in parallel. Forget me, and the model’s thinking with one perspective only.
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

        # This guy combines the heads' outputs. If you skip him, the model can’t consolidate the different perspectives.
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)  # Dropout to prevent overfitting. Too much? We’re losing signal.

    def forward(self, x):
        # Concatenate the output from all heads. Forget this and we lose all the diversity of perspectives.
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        # Combine the result and apply dropout. Forget me and the model might overfit or not learn well.
        out = self.dropout(self.proj(out))
        return out

# I'm a feedforward layer, transforming the attention output. Forget me and the model would be stuck on attention alone.
class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        # A simple 2-layer MLP. No non-linearity here = no complex learning.
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),  # Expanding the dimensionality for more capacity.
            nn.ReLU(),                      # ReLU activation to introduce non-linearity.
            nn.Linear(4 * n_embd, n_embd),  # Bring it back down to original embedding size.
            nn.Dropout(dropout),             # Dropout again to avoid overfitting.
        )

    def forward(self, x):
        return self.net(x)  # Process input through the network.

# Transformer block! We apply attention, feedforward, and residual connections. If I fail, the whole architecture collapses.
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head  # Split the embedding across heads.
        self.sa = MultiHeadAttention(n_head, head_size)  # Multi-head self-attention.
        self.ffwd = FeedFoward(n_embd)  # Feedforward layer.
        self.ln1 = nn.LayerNorm(n_embd)  # Layer normalization for stability.
        self.ln2 = nn.LayerNorm(n_embd)  # Another layer norm. Without normalization, training would be chaotic.

    def forward(self, x):
        x = x + self.sa(self.ln1(x))  # Residual connection after attention.
        x = x + self.ffwd(self.ln2(x))  # Residual connection after feedforward.
        return x

# The main character! Our language model. Forget this and nothing happens.
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Embedding table for tokens. If I’m wrong, the model can’t learn meaningful representations.
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)

        # Embedding table for positions. Without me, the model won’t know the order of tokens.
        self.position_embedding_table = nn.Embedding(block_size, n_embd)

        # Stack multiple transformer blocks. If you skip me, your model is way too shallow.
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])

        # Final layer normalization. Skipping this means more instability during training.
        self.ln_f = nn.LayerNorm(n_embd)

        # Final projection layer to logits. Without this, you can’t make predictions!
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape  # Batch size and sequence length.

        # Get token embeddings. Without these, the model’s just guessing random outputs.
        tok_emb = self.token_embedding_table(idx)

        # Get position embeddings so we know the order of tokens. Without these, the model doesn’t know where anything is.
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))

        # Combine token and position embeddings.
        x = tok_emb + pos_emb

        # Pass through transformer blocks. Forget these and your model is super basic.
        x = self.blocks(x)

        # Apply layer normalization. Skipping this would make training unstable.
        x = self.ln_f(x)

        # Project to logits. If we skip this, there’s no way to predict the next tokens.
        logits = self.lm_head(x)

        # If we’re training (targets are given), calculate the loss.
        if targets is not None:
            B, T, C = logits.shape  # Batch size, sequence length, and number of classes (vocab size).

            # Reshape logits and targets to calculate cross-entropy loss properly. Without reshaping, the loss computation fails.
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)

            # Calculate cross-entropy loss. Forget this and we’re not learning!
            loss = F.cross_entropy(logits, targets)
        else:
            loss = None

        return logits, loss

    # This function generates new text by sampling one token at a time. Forget this and, well, no text generation.
    def generate(self, idx, max_new_tokens):
        # We’ll generate tokens autoregressively, one at a time. Without this loop, we can’t generate long sequences.
        for _ in range(max_new_tokens):
            # Limit context to block_size, otherwise you’ll blow up memory.
            idx_cond = idx[:, -block_size:]

            # Get logits for the next token. No logits = no predictions.
            logits, _ = self(idx_cond)

            # Focus on the logits for the last token. Forget this and you’re sampling from the wrong token!
            logits = logits[:, -1, :]

            # Apply softmax to convert logits to probabilities. No softmax = random noise output.
            probs = F.softmax(logits, dim=-1)

            # Sample the next token based on probabilities. No sampling = no text generation.
            idx_next = torch.multinomial(probs, num_samples=1)

            # Append the new token to the sequence. Forget this and we can’t continue generating.
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

# Instantiate the model and move it to the specified device. No model, no training!
model = BigramLanguageModel()
model = model.to(device)

# AdamW optimizer to adjust weights. Skip this and no learning will happen!
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop – this is where the magic happens. Without this, your model will just sit there doing nothing.
for iter in range(max_iters):
    # Every eval_interval, we check how the model's doing on train and val sets. No eval = no idea how well you're doing!
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"Step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # Get a batch of training data. No batches = no gradient updates!
    xb, yb = get_batch('train')

    # Forward pass: get logits and compute loss. If you skip this, you’ll have no idea what to improve.
    logits, loss = model(xb, yb)

    # Zero out gradients from the previous step. Without this, gradients would accumulate and ruin everything.
    optimizer.zero_grad(set_to_none=True)

    # Backpropagation: compute gradients. No gradients = no learning.
    loss.backward()

    # Update model parameters based on gradients. No optimizer step = no weight adjustments.
    optimizer.step()

# Let’s generate some text! This is the reward after all the training.
context = torch.zeros((1, 1), dtype=torch.long, device=device)  # Start from the zero token.
generated_text = model.generate(context, max_new_tokens=500)  # Generate 500 new tokens.
print(decode(generated_text[0].tolist()))  # Turn the output numbers back into text and print it!


Step 0: train loss 4.4116, val loss 4.4022
Step 100: train loss 2.6568, val loss 2.6670
Step 200: train loss 2.5090, val loss 2.5059
Step 300: train loss 2.4196, val loss 2.4338
Step 400: train loss 2.3504, val loss 2.3566
Step 500: train loss 2.2965, val loss 2.3129
Step 600: train loss 2.2410, val loss 2.2500
Step 700: train loss 2.2057, val loss 2.2191
Step 800: train loss 2.1633, val loss 2.1864
Step 900: train loss 2.1244, val loss 2.1510
Step 1000: train loss 2.1038, val loss 2.1308
Step 1100: train loss 2.0707, val loss 2.1197
Step 1200: train loss 2.0377, val loss 2.0800
Step 1300: train loss 2.0268, val loss 2.0650
Step 1400: train loss 1.9918, val loss 2.0356
Step 1500: train loss 1.9697, val loss 2.0293
Step 1600: train loss 1.9645, val loss 2.0499
Step 1700: train loss 1.9404, val loss 2.0129
Step 1800: train loss 1.9095, val loss 1.9951
Step 1900: train loss 1.9067, val loss 1.9855
Step 2000: train loss 1.8854, val loss 1.9948
Step 2100: train loss 1.8727, val loss 1.9766
